# Linear VMM (Vessel Maneuvering Model) for wPCC using brix coefficients

# Purpose
Implementing according to:
Matusiak, Jerzy. Dynamics of a Rigid Ship. Aalto University, 2017. https://aaltodoc.aalto.fi:443/handle/123456789/24408.

# Methodology
Define the problem using SymPy

# Setup

In [ ]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import numpy as np
import os
import matplotlib.pyplot as plt

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl
from vessel_manoeuvring_models import symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.linear_vmm_equations import *

import vessel_manoeuvring_models.linear_vmm_equations as eq
import vessel_manoeuvring_models.models.linear_vmm as model
from vessel_manoeuvring_models.visualization.plot import track_plot
import vessel_manoeuvring_models.nonlinear_vmm_equations as nonlinear_vmm_equations


## Load test

In [ ]:
df_runs = mdl.runs()

In [ ]:
#id=22773
#id=22616
#id=22774
id=22770


df, units, meta_data = mdl.load(id=id, dir_path='../data/processed/kalman')
df['U'] = np.sqrt(df['u']**2 + df['v']**2)
df['beta'] = -np.arctan2(df['v'],df['u'])
df = df.iloc[100:-100].copy()
df.index = df.index.total_seconds()
df.index-=df.index[0]
df.sort_index(inplace=True)

In [ ]:
df.head()

In [ ]:
meta_data['rho']=1000
meta_data['mass'] = meta_data['Volume']*meta_data['rho']
meta_data.dropna()

In [ ]:
from vessel_manoeuvring_models.visualization.plot import track_plot
fig,ax=plt.subplots()
#fig.set_size_inches(10,10)
track_plot(df=df, lpp=meta_data.lpp, x_dataset='x0', y_dataset='y0',  psi_dataset='psi', beam=meta_data.beam, ax=ax);

# Ship parameters

In [ ]:
T_ = (meta_data.TA + meta_data.TF)/2
L_ = meta_data.lpp
m_ = meta_data.mass
rho_ = meta_data.rho
B_ = meta_data.beam
CB_ = m_/(T_*B_*L_*rho_)
I_z_ = m_*meta_data.KZZ**2

ship_parameters = {
        'T' : T_,
        'L' : L_,
        'CB' :CB_,
        'B' : B_,
        'rho' : rho_,
        'x_G' : 0,  # motions are expressed at CG
        'm' : m_,
        'I_z': I_z_, 
        'volume':meta_data.Volume,
    }

ps = prime_system.PrimeSystem(**ship_parameters)
ship_parameters_prime = ps.prime(ship_parameters)

# Brix parameters

In [ ]:
def calculate_prime(row, ship_parameters):
    return run(function=row['brix_lambda'], inputs=ship_parameters)

mask = df_parameters['brix_lambda'].notnull()
df_parameters.loc[mask,'prime'] = df_parameters.loc[mask].apply(calculate_prime, ship_parameters=ship_parameters, axis=1)
df_parameters.loc['Ydelta','prime'] = 0.005  # Just guessing
df_parameters.loc['Ndelta','prime'] = -df_parameters.loc['Ydelta','prime']/2  # Just guessing
df_parameters['prime'].fillna(0, inplace=True)

In [ ]:
df_parameters.head()

In [ ]:
ship_parameters

In [ ]:
interesting = [
    'u', 
    'v', 
    'r',
    'U',
    'x0',
    'y0',
    'psi',
    'delta',
    'beta',
]
df_ = df[interesting].copy()
df_ = ps.prime(df_, U=df['U'])

In [ ]:
df_.head()

<a id='primetime'></a>

In [ ]:
t = df_.index
t_prime = ps._prime(t, unit='time', U=df['U'])

fig,ax=plt.subplots()
ax.plot(t, t_prime)
ax.set_xlabel('$t$')
ax.set_ylabel("$t'$");


## Prime simulation
<a id='primesimulation'></a>

In [ ]:
interesting = [
    'u', 
    'v', 
    'r',
    'U',
    'x0',
    'y0',
    'psi',
    'delta',
    'beta',
]
df_ = df[interesting].copy()
df_ = ps.prime(df_, U=df['U'])
df_.index=t_prime


#t = df_.index
#control = df_[['delta']]
#control = {'delta':0,}


parameters = dict(df_parameters['prime'])

df_0 = df_.iloc[0]
y0 = {
    'u' : df_0['u'], 
    'v' : df_0['v'],
    'r' : df_0['r'],
    'x0' : df_0['x0'],
    'y0' : df_0['y0'],
    'psi' : df_0['psi'],
    }

#solution = model.simulate(y0=y0, t=t, df_parameters=df_parameters, df_ship_parameters=df_ship_parameters, control=control, rtol=1e-3, atol=1e-3)
result = model.simulator.simulate(df_=df_, parameters=parameters, ship_parameters=ship_parameters_prime, 
                                  control_keys=['delta'])
result.plot_compare()

In [ ]:
result.result

### Go back to SI

In [ ]:
t_prime = result.result.index
t = df.index
fig,ax=plt.subplots()
ax.plot(t,t_prime)

In [ ]:
U_ = t_prime/t*ship_parameters['L']
fig,ax=plt.subplots()
ax.plot(t,U_)
df.plot(y='U', ax=ax)

In [ ]:
#result_SI = ps.unprime(result.result, U=df['U'].values)
result_SI = ps.unprime(result.result, U=U_)
result_SI.index = df.index

In [ ]:
fig,ax = plt.subplots()
track_plot(df=df, lpp=ship_parameters['L'], beam=ship_parameters['B'],ax=ax, label='model test')
track_plot(df=result_SI, lpp=ship_parameters['L'], beam=ship_parameters['B'],ax=ax, label='simulation', color='green')
ax.legend()

for key in result_SI:
    fig,ax = plt.subplots()
    df.plot(y=key, label='model test', ax=ax)
    result_SI.plot(y=key, label='simulation', ax=ax)
    ax.set_ylabel(key)

### Simulation with SI states and prime parameters

In [ ]:
interesting = [
    'u', 
    'v', 
    'r',
    'U',
    'x0',
    'y0',
    'psi',
    'delta',
    'beta',
]
df_ = df[interesting].copy()
parameters = dict(df_parameters['prime'])

df_0 = df_.iloc[0]
y0 = {
    'u' : df_0['u'], 
    'v' : df_0['v'],
    'r' : df_0['r'],
    'x0' : df_0['x0'],
    'y0' : df_0['y0'],
    'psi' : df_0['psi'],
    }

result = model.simulator.simulate(df_=df_, parameters=parameters, ship_parameters=ship_parameters, 
                                  control_keys=['delta'], primed_parameters=True,prime_system=ps)

In [ ]:
result.plot_compare()

## Compare simulation approaches

In [ ]:
fig,ax = plt.subplots()
track_plot(df=result.result, lpp=ship_parameters['L'], beam=ship_parameters['B'],ax=ax, label='primed parameters')
track_plot(df=result_SI, lpp=ship_parameters['L'], beam=ship_parameters['B'],ax=ax, label='simulation', color='green')
ax.legend()

for key in result_SI:
    fig,ax = plt.subplots()
    result.result.plot(y=key, label='primed parameters', ax=ax)
    result_SI.plot(y=key, label='simulation', ax=ax)
    ax.set_ylabel(key)